In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread
import glob
from pandastable import Table, TableModel # Importar clases de pandastable

### Subir reporte danilo

In [2]:
archivo = None
while True:
    archivo=funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            
            #dbf = dbfread.DBF(ruta_archivo)
            monotributo = pd.read_excel(ruta_archivo, sheet_name=0)
            print(monotributo)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/liquidaciones/cruce_contribuciones_para_distribuir/julio a septiembre/errores transferencias de aportes todas juntas desde el 01-05-2023 al 14-0...-2023.xlsx' mode='r' encoding='cp1252'>
              cuit  cuil                                    descripcion  \
0      27286249766   NaN  401: Contrib. Patr. sin Empl. para distribuir   
1      27286249766   NaN  401: Contrib. Patr. sin Empl. para distribuir   
2      27286249766   NaN  401: Contrib. Patr. sin Empl. para distribuir   
3      27286249766   NaN  401: Contrib. Patr. sin Empl. para distribuir   
4      27286249766   NaN  401: Contrib. Patr. sin Empl. para distribuir   
...            ...   ...                                            ...   
26280  33585878109   NaN  401: Contrib. Patr. sin Empl. para distribuir   
26281  30590892544   NaN  401: Contrib. Patr. sin Empl. para distribuir   
26282  30676122601   NaN  401: Contrib. Patr. sin Empl. para distribuir   
26283  3068175

In [3]:
monotributo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26285 entries, 0 to 26284
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   cuit         26285 non-null  int64         
 1   cuil         0 non-null      float64       
 2   descripcion  26285 non-null  object        
 3   obrasocial   0 non-null      float64       
 4   periodo      26285 non-null  int64         
 5   sueldo       26285 non-null  int64         
 6   aporte       26285 non-null  int64         
 7   contri       26285 non-null  float64       
 8   otrosconc    26285 non-null  int64         
 9   fecha        26285 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(5), object(1)
memory usage: 2.0+ MB


###  Importar padron total gecros monotrbutistas

In [4]:
#cargar primero el padron de sidefi
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            padron = pd.read_excel(ruta_archivo)
            #padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='G:/Mi unidad/osoetsylra/liquidaciones/cruce_contribuciones_para_distribuir/LIQUIDACIONES2.xlsx' mode='r' encoding='cp1252'>
       OS_NOMBRE          CUIL SEXO      NOMPLAN PARENTESCO  \
0      EVP (TIS)           NaN    F  Desregulado     HIJO/A   
1      EVP (TIS)           NaN    F  Desregulado     HIJO/A   
2      EVP (TIS)           NaN    F  Desregulado    TITULAR   
3      EVP (TIS)           NaN    F  Desregulado    TITULAR   
4      EVP (TIS)           NaN    F  Desregulado    TITULAR   
...          ...           ...  ...          ...        ...   
66921    VISITAR  2.756850e+10    F    Afiliados     HIJO/A   
66922    VISITAR  2.758200e+10    F    Afiliados     HIJO/A   
66923    VISITAR  2.758853e+10    F    Afiliados     HIJO/A   
66924    VISITAR  2.793704e+10    F    Afiliados    TITULAR   
66925    VISITAR  2.795855e+10    F    Afiliados    TITULAR   

                APELLIDO              NOMBRES                  TIPOBEN_NO  
0                 

In [5]:
padron['CUIL'] = pd.to_numeric(padron['CUIL'], errors='coerce').fillna(0).astype('Int64')
padron['CUIL'].head()

0    0
1    0
2    0
3    0
4    0
Name: CUIL, dtype: Int64

#### Cruce de archivos! 

In [6]:
liquidaciones_monot = pd.merge(monotributo, padron, left_on= 'cuit', right_on='CUIL', how='inner', indicator=True)
liquidaciones_monot["Razón Social"]=" "
liquidaciones_monot['periodo'] = liquidaciones_monot['periodo'].astype(str).apply(lambda x: "20" + x)
liquidaciones_monot['periodo'] = liquidaciones_monot['periodo'].astype(int)
liquidaciones_monot.columns

Index(['cuit', 'cuil', 'descripcion', 'obrasocial', 'periodo', 'sueldo',
       'aporte', 'contri', 'otrosconc', 'fecha', 'OS_NOMBRE', 'CUIL', 'SEXO',
       'NOMPLAN', 'PARENTESCO', 'APELLIDO', 'NOMBRES', 'TIPOBEN_NO', '_merge',
       'Razón Social'],
      dtype='object')

In [7]:
print(liquidaciones_monot["periodo"])

0        202201
1        202206
2        202210
3        202212
4        202301
          ...  
19442    202205
19443    202206
19444    202207
19445    202208
19446    202307
Name: periodo, Length: 19447, dtype: int32


In [8]:
aportes_a_importar = liquidaciones_monot[["cuit",
"Razón Social",
"periodo",
"fecha",
"CUIL",
"APELLIDO",
"NOMBRES",
"sueldo",
"aporte",
"contri",
"otrosconc"
]]

In [9]:
path = funciones.seleccionar_directorio()
liquidaciones_monot.to_excel(path + '\cruce_erroresoctubre_raw.xlsx', index=False)
aportes_a_importar.to_excel(path + '\cruce_erroresocutubre.xls', index = False, engine='xlwt', header = False)

C:\Users\zickd\AppData\Local\Temp\ipykernel_3136\4046240144.py:3: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  aportes_a_importar.to_excel(path + '\cruce_erroresocutubre.xls', index = False, engine='xlwt', header = False)


In [48]:
liquidaciones_monot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8672 entries, 0 to 8671
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   cuit         8672 non-null   int64         
 1   cuil         0 non-null      float64       
 2   descripcion  8672 non-null   object        
 3   obrasocial   0 non-null      float64       
 4   periodo      8672 non-null   int64         
 5   sueldo       8672 non-null   int64         
 6   aporte       8672 non-null   int64         
 7   contri       8672 non-null   float64       
 8   otrosconc    8672 non-null   int64         
 9   fecha        8672 non-null   datetime64[ns]
 10  OS_NOMBRE    6400 non-null   object        
 11  CUIL         6400 non-null   Int64         
 12  NOMPLAN      6400 non-null   object        
 13  PARENTESCO   6400 non-null   object        
 14  FECHA_BAJA   1982 non-null   object        
 15  EMP_NOMBRE   6394 non-null   object        
 16  APELLI

: 